In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import re
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import operator
from konlpy.tag import Okt
Okt = Okt()
import nltk
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
from math import log
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel('wanted_crawling.xlsx')

In [3]:
def checking_null(data):
    print(data.isnull().sum())





#불필요 행 제거 및 데이터 양 확인
def del_null(data):
    print('기존 데이터 개수는 ',len(data))
        
    data=data.drop_duplicates()
    print('중복제거 데이터 개수는 ',len(data))
        
    data=data.dropna(subset=['직무','유사직무'])
    data = data.reset_index(drop=True)
    print('직무,유사직무 결측값제거 데이터 개수',len(data))
    
    
    ## 이상한 에러값(길이10이하, \n만 있는 애들) 공백으로 만들기
    for i in tqdm_notebook(range(len(data))):
        if len(str(data['주요업무'][i])) < 10 and '\\n' in str(data['주요업무'][i]):       
            data.loc[i,['주요업무']] = ''
        if len(str(data['자격요건'][i])) < 10 and '\\n' in str(data['자격요건'][i]):
            data.loc[i,['자격요건']] = ''
        if len(str(data['우대사항'][i])) < 10 and '\\n' in str(data['우대사항'][i]):
            data.loc[i,['우대사항']] = ''
    data = data.reset_index(drop=True)
    
    
    ## '주요업무', '자격요건', '우대사항' 전부 글자수 10미만인거 제거
    idx = []
    for i in tqdm_notebook(range(len(data))):
        if len(str(data['주요업무'][i])) < 10 and len(str(data['자격요건'][i])) < 10 and len(str(data['우대사항'][i])) < 10:
            idx.append(i)
    data = data.drop(idx)
    data = data.reset_index(drop=True)
    print('완료된 데이터 개수',len(data))
    return data
    
    

       
## 중국어,일본어 10개이상 포함된 애들 빼기    
def del_chinese_japanese(data):
    index=[]
    for i in range(len(data)):
        temp = str(data['주요업무'][i]) + str(data['자격요건'][i]) + str(data['우대사항'][i])
        x=re.findall(r'[\u4e00-\u9fff]+', temp)
        y=re.findall(r'[\u3040-\u30FC]+', temp)
        if len(list(set(x)))>10 or len(list(set(y)))>10:
            index.append(i)

    data = data.drop(index)
    data = data.reset_index(drop=True)
    return data






## 리스트 원소들 앞뒤 여백제거 함수
def strip_(list):
    temp=[]
    for j in list:
        if len(str(j)) > 1:
            temp.append(j.strip())
    return temp







## 데이터 내 원소들 \t와 • 제거, 앞뒤 여백 제거
def del_error(data):
    for i in tqdm(range(len(data))):
        try:
            data.loc[i,'주요업무']='\\n'.join(strip_(data.loc[i,'주요업무'].replace('\\t','').replace('•','').split('\\n')))        
        except:
            pass
        try:
            data.loc[i,'자격요건']='\\n'.join(strip_(data.loc[i,'자격요건'].replace('\\t','').replace('•','').split('\\n')))        
        except:
            pass
        try:
            data.loc[i,'우대사항']='\\n'.join(strip_(data.loc[i,'우대사항'].replace('\\t','').replace('•','').split('\\n')))        
        except:
            pass
    return data



def extract2to1(data):
    for i in tqdm(range(len(data))):
        print(i,'시작')
        temp=list(data['유사직무'][i].split('\",\"'))
        if len(temp)%2 == 1 and ('Software Engineer' in temp):
            temp.remove('Software Engineer')
        if len(temp)%2 == 1 and ('Performance Marketing Manager' in temp):
            temp.remove('Performance Marketing Manager')
        if len(temp)%2 == 1 and ('Administrative Assistant' in temp):
            temp.remove('Administrative Assistant')    
        temp = temp[1::2]
        x=0
        while x<len(temp):
            if x+1 == len(temp):
                temp[x] = str('\"'+temp[x])
            else:
                temp[x] = str('\"'+temp[x]+'\"')
            x+=1

        data.loc[i,'유사직무'] = ',  '.join(temp)
        print(temp)
        print(i,'끝')
        
    return data




def counting_job(data):
    a=list(data['유사직무'])
    duplicated=[]
    for i in tqdm(range(len(data))):
        duplicated.extend(strip_(data['유사직무'][i].split(',  ')))
    print('기존 유사직무 총 개수(중복포함) :',len(duplicated))
    not_duplicated=set(duplicated)
    print('기존 유사직무 개수(중복제거) :',len(not_duplicated))
    not_duplicated = list(not_duplicated)
    
    return duplicated, not_duplicated




def counting_data_per_job(data):    
    sim_job={}
    for i in range(len(not_duplicated)):
        sim_job[not_duplicated[i]]=duplicated.count(not_duplicated[i])
    dict_= sorted(sim_job.items(),key=operator.itemgetter(1), reverse=True)
    return dict_



  
def rearrange(data):
    new_data = pd.DataFrame(columns = ['직무','출연횟수','주요업무','자격요건','우대사항'])
   
    for i in tqdm(range(len(not_duplicated))):
        temp1=[]
        temp2=[]
        temp3=[]
        count = 0
        for j in range(len(data)):
            if not_duplicated[i] in data['유사직무'][j]:
                count +=1
                temp1.extend(str(data['주요업무'][j]).split('\\n'))
                temp2.extend(str(data['자격요건'][j]).split('\\n'))
                temp3.extend(str(data['우대사항'][j]).split('\\n'))

        new_data=new_data.append({'직무' : not_duplicated[i],'출연횟수':count, '주요업무' : temp1, '자격요건' : temp2, '우대사항' : temp3} , ignore_index=True)

    new_data=new_data.sort_values(by=['출연횟수','직무'],ascending = False)  
    return new_data






def merge(kor,eng,ans):
    index_eng = 0
    index_kor = 0
    answer=[]        

    if len(eng) == 0:
        answer = kor

    else:
        while True:
            a = kor[index_kor]
            b = eng[index_eng]
            
            if ans.find(a) < ans.find(b):
                answer.append(a)
                ans = ans[ans.find(a)+len(a):]
                index_kor += 1
                
            else:
                answer.append(b)
                ans = ans[ans.find(b)+len(b):]
                index_eng += 1
                
            if index_eng == len(eng) or index_kor == len(kor):
                
                if index_kor != len(kor):
                
                    while index_kor != len(kor):                    
                        answer.append(kor[index_kor])
                        index_kor += 1

                else:
                    while index_eng != len(eng):
                        answer.append(eng[index_eng])
                        index_eng +=1
                break

    return ' '.join(answer)
    
    
    
    
    
    
    
def nominalization(data):

    for i in range(len(data)):
        if str(data['직무'][i])[0] == '\"':
            data.loc[i,'직무'] = str(data['직무'][i])[1:]
        if str(data['직무'][i])[-1] == '\"':
            data.loc[i,'직무'] = str(data['직무'][i])[:-1]
    
    
    data = data.drop(data.index[340],axis=0)
    data = data.reset_index(drop=True)
    
    data = data.drop(data.index[349],axis=0)
    data = data.reset_index(drop=True)
    
    
    
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+')
    english = re.compile('[^a-zA-Z]+')

    for i in tqdm(range(len(data))): 
        kor = Okt.nouns(hangul.sub(' ', str(data['주요업무'][i])))
        eng = english.sub(' ', str(data['주요업무'][i])).lower().split()
        ans =str(data['주요업무'][i]).lower()
        data.loc[i,'주요업무'] = merge(kor,eng,ans)
        

        kor = Okt.nouns(hangul.sub(' ', str(data['자격요건'][i])))
        eng = english.sub(' ', str(data['자격요건'][i])).lower().split()
        ans =str(data['자격요건'][i]).lower()
        data.loc[i,'자격요건'] = merge(kor,eng,ans)
        

        kor = Okt.nouns(hangul.sub(' ', str(data['우대사항'][i])))
        eng = english.sub(' ', str(data['우대사항'][i])).lower().split()
        ans =str(data['우대사항'][i]).lower()
        data.loc[i,'우대사항'] = merge(kor,eng,ans)
        
        
    return data





def about_carrer(data):
    for i in range(len(data)):
        if str(data['직무'][i])[0] == '\"':
            data.loc[i,'직무'] = str(data['직무'][i])[1:]
        if str(data['직무'][i])[-1] == '\"':
            data.loc[i,'직무'] = str(data['직무'][i])[:-1]

    data = data.drop(data.index[340],axis=0)
    data = data.reset_index(drop=True)
    data = data.drop(data.index[349],axis=0)
    data = data.reset_index(drop=True)

    data=data.sort_values(by=['출연횟수','직무'],ascending = False)
    data = data.reset_index(drop=True)
    
    
    
    career = []
    career_count=[]
    mean = []
    for i in range(len(data)):    
        temp = []
        text = ' '.join(data['자격요건'][i])
        num = re.findall('\d+',text)
        for j in num:
            try:
                if text[text.find(j) + len(str(j))] == '년' and text[text.find(j) + len(str(j)):text.find(j) + len(str(j))+2] != '년제' and int(j)<30: 
                    text = text[text.find(j) + len(str(j))+1:]
                    temp.append(j)
            except:
                pass
        if len(temp) == 0:
            career.append('nan')
            career_count.append(int(0))
        else:
            career.append(round(sum(map(int,temp))/len(temp),2))
            career_count.append(len(temp))


    career_count_ = []
    for i in range(len(career_count)):
        if int(career_count[i]) / int(data['출연횟수'][i]) * 100 > 100:
            career_count_.append('100%') 
        else:
            career_count_.append(str(int(career_count[i]) / int(data['출연횟수'][i]) * 100) +'%')
    return career,career_count_






def tf(t, d):
    return d.count(t)


def idf(t,docs):
    df = 0
    N = len(docs)
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))


def tfidf(t, d,docs):
    return tf(t,d)* idf(t,docs)



def dtm(): 
    result = []
    for j in range(len(vocab)):
        t = vocab[j]
        result.append(idf(t))

    idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
    return idf_



def final(docs):
    N = len(docs)
    vocab = list(set(w for doc in docs for w in doc.split())) 
    vocab.sort()

    result = []
    for i in range(N):
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]

            result[-1].append(tfidf(t,d,docs))

    tfidf_ = pd.DataFrame(result, columns = vocab)
    return tfidf_    






def final(docs):
    N = len(docs)
    vocab = list(set(w for doc in docs for w in doc.split())) 
    vocab.sort()

    result = []
    for i in range(N):
        result.append([])
        d = docs[i]
        for j in range(len(vocab)):
            t = vocab[j]

            result[-1].append(tfidf(t,d,docs))

    tfidf_ = pd.DataFrame(result, columns = vocab)
    return tfidf_



def extract_keyword_tfidf(data):
    temp1=list(data['주요업무'])
    temp2=list(data['자격요건'])
    temp3=list(data['우대사항'])
    
    x= final(temp1)
    x_=[]
    for i in tqdm(range(len(temp1))):
        if len(sorted(x.loc[i].items(),key = lambda item:item[1],reverse = True)) > 100:
            temp= dict(sorted(x.loc[i].items(),key = lambda item:item[1],reverse = True)[:100]).keys()
        else:
            temp= dict(sorted(x.loc[i].items(),key = lambda item:item[1],reverse = True)).keys()
        x_.append(str(list(temp))[1:-1])
    data['TF_IDF기준 주요업무 상위100keywords'] = x_
    
    
    y= final(temp2)
    y_=[]
    for i in tqdm(range(len(temp2))):
        if len(sorted(y.loc[i].items(),key = lambda item:item[1],reverse = True)) > 100:
            temp= dict(sorted(y.loc[i].items(),key = lambda item:item[1],reverse = True)[:100]).keys()
        else:
            temp= dict(sorted(y.loc[i].items(),key = lambda item:item[1],reverse = True)).keys()
        y_.append(str(list(temp))[1:-1])
    data['TF_IDF기준 자격요건 상위100keywords'] = y_
    
    
    
    z= final(temp3)
    z_=[]
    for i in tqdm(range(len(temp3))):
        if len(sorted(z.loc[i].items(),key = lambda item:item[1],reverse = True)) > 100:
            temp= dict(sorted(z.loc[i].items(),key = lambda item:item[1],reverse = True)[:100]).keys()
        else:
            temp= dict(sorted(z.loc[i].items(),key = lambda item:item[1],reverse = True)).keys()
        z_.append(str(list(temp))[1:-1])
    data['TF_IDF기준 우대사항 상위100keywords'] = z_
    
    return data






def Doc2vec_embeddnig(data):
    temp = data['주요업무'] + data['자격요건'] + data['우대사항']
    tagged_text = [TaggedDocument(words=word_tokenize(d.lower()), tags=[str(i)]) for i,d in enumerate(temp)]
    
    
    max_epochs = 100
    vec_size = 16
    alpha = 0.025

    model = Doc2Vec(size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=5,
                    dm =1)

    model.build_vocab(tagged_text)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_text,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha

    model.save("d2v.model")
    print("Model Saved")



    model= Doc2Vec.load("d2v.model")

    vector_lis = []
    for i in range(len(temp)):
        v=model.infer_vector(word_tokenize(temp[i].lower()))
        vector_lis.append(v)
        
    return vector_lis






def visualization(data,list_):
    text_standard = StandardScaler().fit_transform(vector_lis)
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(text_standard)
    
    df = pd.DataFrame(data = principalComponents, columns = ['principal_component_1', 'principal_component_2'],index=data['직무'])
    
    
    
    plt.figure(figsize=(50, 50))
    ax=sns.scatterplot(x='principal_component_1',y='principal_component_2',data=df) 
    plt.title('시각화')

    for i in range(len(df)):
        ax.text(float(df['principal_component_1'][i]) - 0.5, float(df['principal_component_2'][i]) + 0.05, df.index[i])

In [ ]:
checking_null(data)
data = del_null(data)
data = del_chinese_japanese(data)
data = del_error(data)
data = extract2to1(data)
duplicated, not_duplicated = counting_job(data)
print(duplicated)
print(not_duplicated)
sub_explain = counting_data_per_job(data)
print(sub_explain)
data = rearrange(data)
data_sub = data

career, career_count = about_carrer(data_sub)
data = nominalization(data)

data['요구된 평균경력'] = career
data['경력 요구 비율'] = career_count

data = data[data['출연횟수']>9]

data = extract_keyword_tfidf(data)

vector_lis = Doc2vec_embeddnig(data)
visualization(data,vector_lis)

Unnamed: 0      0
주소번호            0
직무             63
유사직무          356
주요업무           31
자격요건           46
우대사항          279
dtype: int64
기존 데이터 개수는  50926
중복제거 데이터 개수는  50926
직무,유사직무 결측값제거 데이터 개수 50567


<ipython-input-3-6aaab0e6dc84>:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(data))):


<ipython-input-3-6aaab0e6dc84>:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(data))):



완료된 데이터 개수 50500
